In [45]:
import json

# read the file
def corruptables(jsonfile):
    to_corrupt = []
    with open(jsonfile) as f:
        out = [(x) for x in json.load(f)]
        for o in out:
            coco = []
            for oo in o:
                ooo = (oo[0], oo[1])
                coco.append(ooo)
            to_corrupt.append(coco)

    return to_corrupt

In [47]:
import os
import re

with open('sentences_to_corrupt.txt', 'r') as f:
    file = f.read()
    split = file.split('.')

proper_split_0 = []    
for s in split:
    if '?' in s:
        ss = s.split('?')
        for se in ss:
            proper_split_0.append(se + '?')
    else:
        proper_split_0.append(s)

proper_split = []    
for s in proper_split_0:
    if '!' in s:
        ss = s.split('!')
        for se in ss:
            proper_split.append(se + '!')
    else:
        proper_split.append(s)
        
for p in proper_split:
    if len(p.split()) <= 2: #removing sentences of 1 or 2 words because with 1 word, no switch between 2 words can be made,
                            #and 2 not representative enough - there's no such case in the example data
        proper_split.remove(p)
    else:
        pass

print('Number of sentences:', len(proper_split))

Number of sentences: 23004


In [57]:
to_corrupt_3000 = corruptables('sentences_to_corrupt_3000.json')
to_corrupt_4000 = corruptables('sentences_to_corrupt_4000.json')
to_corrupt_5000 = corruptables('sentences_to_corrupt_5000.json')
to_corrupt_6000 = corruptables('sentences_to_corrupt_6000.json')
to_corrupt_7000 = corruptables('sentences_to_corrupt_7000.json')
to_corrupt_8000 = corruptables('sentences_to_corrupt_8000.json')
to_corrupt_9000 = corruptables('sentences_to_corrupt_9000.json')
to_corrupt_10000 = corruptables('sentences_to_corrupt_10000.json')
to_corrupt_11000 = corruptables('sentences_to_corrupt_11000.json')
to_corrupt_12000 = corruptables('sentences_to_corrupt_12000.json')
to_corrupt_13000 = corruptables('sentences_to_corrupt_13000.json')
to_corrupt_14000 = corruptables('sentences_to_corrupt_14000.json')
to_corrupt_15000 = corruptables('sentences_to_corrupt_15000.json')
to_corrupt_16000 = corruptables('sentences_to_corrupt_16000.json')
to_corrupt_17000 = corruptables('sentences_to_corrupt_17000.json')
to_corrupt_18000 = corruptables('sentences_to_corrupt_18000.json')
to_corrupt_19000 = corruptables('sentences_to_corrupt_19000.json')
to_corrupt_20000 = corruptables('sentences_to_corrupt_20000.json')
to_corrupt_21000 = corruptables('sentences_to_corrupt_21000.json')
to_corrupt_22000 = corruptables('sentences_to_corrupt_22000.json')
to_corrupt_23000 = corruptables('sentences_to_corrupt_23000.json')

total = to_corrupt_3000 + to_corrupt_4000 + to_corrupt_5000 + to_corrupt_6000 + to_corrupt_7000 + to_corrupt_8000 + to_corrupt_9000 + to_corrupt_10000 + to_corrupt_11000 + to_corrupt_12000 + to_corrupt_13000 + to_corrupt_14000 + to_corrupt_15000 + to_corrupt_16000 + to_corrupt_17000 + to_corrupt_18000 + to_corrupt_19000 + to_corrupt_20000 + to_corrupt_21000 + to_corrupt_22000 + to_corrupt_23000

print(len(total))

def remove_duplicates(l):
    f = []
    for ll in l:
        if ll not in f:
            f.append(ll)
    return f

all_corrupt = remove_duplicates(total)
print(len(all_corrupt))
testy = all_corrupt[:100]

20369
20275


In [58]:
def AB_rules(sent): #sent is of type list
    """
    AB (Adverb) RULES:
        1. (og) VB - AB --> (corrupted) AB - VB 
        2. (og) AB - VB --> (corrupted) VB - AB : subordinate clauses, usually involves a SN (att)
                                                  in a previous position, and AB is often a negation (inte)
    """
    AB_corrupted = []
    for t in sent:
        for i, token in enumerate(t):
            if token[0] == 'VB':
                try:
                    if t[t.index(t[i+1])][1] == ':' or t[t.index(t[i+2])][1] == ':' or t[t.index(t[i+3])][1] == ':':
                        trash.append(i)
                    else:
                        try:
                            if t[i-1][0] == 'AB':
                                t_copy = t.copy()
                                ab_i = t.index(t[i-1])
                                t_copy[ab_i], t_copy[i] = t_copy[i], t_copy[ab_i]
                                AB_corrupted.append((t_copy, t))
        #                         print(t_copy, t)
                            elif t[i+1][0] == 'AB':
                                t_copy = t.copy()
                                ab_i = t.index(t[i+1])
                                t_copy[i], t_copy[ab_i] = t_copy[ab_i], t_copy[i]
                                AB_corrupted.append((t_copy, t))
        #                         print(t_copy, t)
                            else:
                                  pass
                        except:
                            pass
                except:
                    pass
            else:
                pass
    return AB_corrupted

In [59]:
def NN_rules(sent):
    """
    NN (Noun) RULES:
        When the verb is in a subordinate clause, and the NN does not act as the object of the VB:
        1. (og) VB - NN --> (corrupted) NN - VB : when something comes before it, such as an AB
        2. (og) VB - DT + NN --> (corrupted) DT + NN - VB ('+' == two word classes go together, as a pack)
        3. (og) VB - PS + NN --> (corrupted) PS + NN - VB
    Conditions that don't allow the previous rules to form:
        - if NN is complemented with a subordinate clause, e.g. "namn som innehåller ord från naturen"
    """
    NN_corrupted = []
    for t in sent:
        for i, token in enumerate(t):
            if token[0] == 'VB':
                try:
                    if t[t.index(t[i+1])][1] == ':' or t[t.index(t[i+2])][1] == ':' or t[t.index(t[i+3])][1] == ':':
                        trash.append(i)
                    else:
                        try:
                            if t[i+1][0] == 'NN':
                                t_copy = t.copy()
                                nn_i = t.index(t[i+1])
                                t_copy[i], t_copy[nn_i] = t_copy[nn_i], t_copy[i]
                                NN_corrupted.append((t_copy, t))
        #                         print(t_copy, t)
                            elif t[i+2][0] == 'NN':
                                if t[i+1][0] == 'PS' or t[i+1][0] == 'DT': #possessive pronouns or determiners
                                    t_copy = t.copy()
                                    nn_i = t.index(t[i+2])
                                    ps_or_dt_i = t.index(t[i+1])
                                    t_copy[i], t_copy[ps_or_dt_i], t_copy[nn_i] = t_copy[ps_or_dt_i], t_copy[nn_i], t_copy[i]
                                    NN_corrupted.append((t_copy, t))
                                elif t[i+1][0] == 'JJ': #adjectives 
                                    t_copy = t.copy()
                                    nn_i = t.index(t[i+2])
                                    jj_i = t.index(t[i+1])
                                    t_copy[i], t_copy[jj_i], t_copy[nn_i] = t_copy[jj_i], t_copy[nn_i], t_copy[i]
                                    NN_corrupted.append((t_copy, t))
                            elif t[i-1][0] == 'NN': #adjectives before
                                if t[i-2][0] == 'JJ':
                                    t_copy = t.copy()
                                    nn_i = t.index(t[i-1])
                                    jj_i = t.index(t[i-2])
                                    t_copy[jj_i], t_copy[nn_i], t_copy[i] = t_copy[i], t_copy[jj_i], t_copy[nn_i]
                                    NN_corrupted.append((t_copy, t))
                            else:
                                  pass
                        except:
                            pass
                except:
                    pass
            else:
                pass
    return NN_corrupted

In [60]:
def PN_rules(sent): 
    """
    PN (Pronoun) RULES:
        In the beginning of sentences:
            1. (og) (AB) VB - PN --> (corrupted) PN - VB 
            2. (og) (HA) PN - VB --> (corrupted) VB - PN
            3. (og) (KN) VB - PN --> (corrupted) PN - VB (subordinating conjunction)
            4. (og) VB - PN --> PN - VB : past 1st half of sentence, usually with subordinate clauses involved (om, när...)
            5. (og) PN - VB --> VB - PN : very beginning of sentences (not super common)
            6. (og) (HP) PN - VB --> VB - PN : frågande/relativ pron
    """
    PN_corrupted = []
    trash = []
    for t in sent:
        for i, token in enumerate(t):
            if token[0] == 'VB':
                try:
                    if t[t.index(t[i+1])][1] == ':' or t[t.index(t[i+2])][1] == ':' or t[t.index(t[i+3])][1] == ':':
                        trash.append(i)
                    else:
                        try:
                            if t[i+1][0] == 'PN':
                                if t[i-1][0] == 'AB' or t[i-1][0] == 'KN': #limiting to cases of subordination
                                    t_copy = t.copy()
                                    pn_i = t.index(t[i+1])
                                    t_copy[i], t_copy[pn_i] = t_copy[pn_i], t_copy[i]
                                    PN_corrupted.append((t_copy, t))
        #                             print(t_copy, t)
                            elif t[i-1][0] == 'PN':
                                t_copy = t.copy()
                                pn_i = t.index(t[i-1])
                                t_copy[pn_i], t_copy[i] = t_copy[i], t_copy[pn_i]
                                PN_corrupted.append((t_copy, t))
        #                         print(t_copy, t)
                            else:
                                  pass
                        except:
                            pass
                except:
                    pass
            else:
                pass
    return PN_corrupted

In [61]:
def PM_rules(sent):
    """
    PM (Proper Noun) RULES:
        When the verb is in a subordinate clause, and the PM does not act as the object of the VB:
        1. (og) VB - PM --> (corrupted) PM - VB : when something comes before it, such as an AB
        2. (og) VB - PM + PM --> (corrupted) PM + PM - VB : the case for name + surname
    """
    PM_corrupted = []
    for t in sent:
        for i, token in enumerate(t):
            if token[0] == 'VB':
                try:
                    if t[t.index(t[i+1])][1] == ':' or t[t.index(t[i+2])][1] == ':' or t[t.index(t[i+3])][1] == ':':
                        trash.append(i)
                    else:
                        try:
                            if t[i+1][0] == 'PM':
                                if t[i+2][0] == 'PM':
                                    t_copy = t.copy()
                                    pm_i = t.index(t[i+1])
                                    pm_2_i = t.index(t[i+2])
                                    t_copy[i], t_copy[pm_i], t_copy[pm_2_i] = t_copy[pm_i], t_copy[pm_2_i], t_copy[i]
                                    PM_corrupted.append((t_copy, t))
                                else:
                                    t_copy = t.copy()
                                    pm_i = t.index(t[i+1])
                                    t_copy[i], t_copy[pm_i] = t_copy[pm_i], t_copy[i]
                                    PM_corrupted.append((t_copy, t))
            #                         print(t_copy, t)
                            else:
                                  pass
                        except:
                            pass
                except:
                    pass
            else:
                pass
    return PM_corrupted

In [62]:
corrupted_AB = AB_rules(all_corrupt)
corrupted_NN = NN_rules(all_corrupt)
corrupted_PN = PN_rules(all_corrupt)
corrupted_PM = PM_rules(all_corrupt)
print(len(corrupted_AB), 'sentences with adverbs corrupted')
print(len(corrupted_NN), 'sentences with nouns corrupted')
print(len(corrupted_PN), 'sentences with pronouns corrupted')
print(len(corrupted_PM), 'sentences with proper nouns corrupted')
# print(corrupted_PN)

9196 sentences with adverbs corrupted
7331 sentences with nouns corrupted
12111 sentences with pronouns corrupted
682 sentences with proper nouns corrupted


In [63]:
def cleaner(a_list):
    final = []
    for sentence in a_list:
        s1, s2 = sentence
        sent1 = ''
        sent2 = ''
        for w_tuple in s1:
            sent1 += w_tuple[1] + ' '
        for w_tuple in s2:
            sent2 += w_tuple[1] + ' '
        final.append((sent1, sent2))
    return final

In [64]:
import random
import csv

corrupted_clean_AB = cleaner(corrupted_AB)
corrupted_clean_NN = cleaner(corrupted_NN)
corrupted_clean_PN = cleaner(corrupted_PN)
corrupted_clean_PM = cleaner(corrupted_PM)

everything = corrupted_clean_AB + corrupted_clean_NN + corrupted_clean_PN + corrupted_clean_PM
random.shuffle(everything)
error_label = 'S-FinV'

absolutely_everything = []
for a in everything:
    o, c = a
    final = (o, c, error_label)
    absolutely_everything.append(final)

with open('S_FinV_error_dataset.csv','w') as f:
    csv_f = csv.writer(f)
    csv_f.writerow(['Original sentence', 'Corrected sentence','Error label'])
    csv_f.writerows(absolutely_everything)

In [65]:
print(len(absolutely_everything))

29320
